In [ ]:
import cv2
import os,dlib,glob
import numpy as np
from skimage import io,transform
import operator

def send_email(img_path,to_list):#发送图片路径，收件人
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.image import MIMEImage
    import smtplib
    import matplotlib.pyplot as plt


    if __name__ == '__main__':
        # 以html格式构建邮件内容
        send_str = '<html><body>'
        send_str += '<center>危险预警-陌生人</center>'
        send_str += '</body></html>'
        pic_path = 'test.png'
        msg = MIMEMultipart()

        # 添加邮件内容
        content = MIMEText(send_str, _subtype='html', _charset='utf8')
        msg.attach(content)

        # 邮件主题
        msg['Subject'] = '陌生人到访'

        # 邮件收、发件人
        user = "2743584180@qq.com"
        msg['To'] = ';'.join(to_list)
        msg['From'] = user
        # 密码（有些邮件服务商在三方登录时需要使用授权码而非密码，比如网易和QQ邮箱）
        passwd = "actvsxsystkpdhag"

        # 构建并添加附件对象
        # 如果不想直接在邮件中展示图片，可以以附件形式添加
        img = MIMEImage(open(img_path, 'rb').read(), _subtype='octet-stream')
        img.add_header('Content-Disposition', 'attachment', filename=pic_path)
        msg.attach(img)

        server = smtplib.SMTP_SSL("smtp.qq.com",port=465)
        server.login(user, passwd)
        server.sendmail(user, to_list, msg.as_string())

def distMap(frame1, frame2):
    frame1_32 = np.float32(frame1)
    frame2_32 = np.float32(frame2)
    diff32 = frame1_32 - frame2_32
    norm32 = np.sqrt(diff32[:,:,0]**2 + diff32[:,:,1]**2 + \
             diff32[:,:,2]**2)/np.sqrt(255**2 + 255**2 + 255**2)
    dist = np.uint8(norm32*255)
    
    mod = cv2.GaussianBlur(dist, (9,9), 0)
    _, thresh = cv2.threshold(mod, 100, 255, 0)
    _, stDev = cv2.meanStdDev(mod)
    return stDev

def get_images_and_labels(faces_folder_path,img_path):#人脸文件夹,需识别的人脸
    # 人脸描述子list
    descriptors = []
    # 对文件夹下的每一个人脸进行:
    # 1.人脸检测
    # 2.关键点检测
    # 3.描述子提取
    candidate = []
    for f in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
        candidate.append(f.split('\\')[1])
        img = io.imread(f)
        if(len(img.shape) != 3):
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        # 1.人脸检测
        dets = detector(img, 1)

        for k, d in enumerate(dets): 
        # 2.关键点检测
            shape = sp(img, d)
        # 3.描述子提取，128D向量
            face_descriptor = facerec.compute_face_descriptor(img, shape)
        # 转换为numpy array
            v = np.array(face_descriptor)  
            descriptors.append(v)

    # 对需识别人脸进行同样处理
    # 提取描述子
    img = io.imread(img_path)
    if(len(img.shape) != 3):
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    dets = detector(img, 1)
    if(len(dets)==0):
        return False
    for k, d in enumerate(dets):
        shape = sp(img, d)
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        d_test = np.array(face_descriptor) 
    
    
    # 计算欧式距离
    dist = []
    for i in descriptors:
        dist_ = np.linalg.norm(i-d_test)
        dist.append(dist_)
    return 1-np.mean(np.array(dist))#正确率
import cv2
# 1.人脸关键点检测器
predictor_path = 'face-model/shape_predictor_68_face_landmarks.dat'
# 2.人脸识别模型
face_rec_model_path = 'face-model/dlib_face_recognition_resnet_model_v1.dat'

# 1.加载正脸检测器
detector = dlib.get_frontal_face_detector()
# 2.加载人脸关键点检测器
sp = dlib.shape_predictor(predictor_path)
# 3. 加载人脸识别模型
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

cap = cv2.VideoCapture(0)
while(1):
    ret,frame = cap.read()
    ret2,frame2=cap.read()
    cv2.imshow('camera',frame)
    #动态检测
    stDev = distMap(frame, frame2)
    
    if stDev < 3:
        gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        font = cv2.FONT_HERSHEY_SIMPLEX
        
        faces = detector(gray)
        if(len(faces)):
            for face in faces:
                left = face.left()
                top = face.top()
                right = face.right()
                bottom = face.bottom()
                if(left<0 or top<0 or right<0 or bottom <0):
                    continue
                imgg = np.copy(gray)
                imgg = imgg[top:bottom,left:right]
                cv2.imwrite('face.jpg',imgg)
                cv2.rectangle(frame, pt1=(left, top), pt2=(right, bottom), color=(0, 255, 0), thickness=4)

        if(os.path.exists('face.jpg')):

            test = cv2.GaussianBlur(frame, (15,15), 0)
            imgzi = cv2.putText(test,'Testing', (50, 100), font, 2.5, (255, 255, 255), 2)
            cv2.imshow('camera',test)

            preds = {'zhang':0,'mi':0,'wang':0}
            preds['zhang'] = get_images_and_labels('image/zhang','face.jpg')
            preds['mi'] = get_images_and_labels('image/mi','face.jpg')
            preds['wang'] = get_images_and_labels('image/wang','face.jpg')
            preds = sorted(preds.items(),key = operator.itemgetter(1),reverse = True)
            if(preds[0][1]>0.65):
                T = cv2.GaussianBlur(frame, (15,15), 0)
                imgzi = cv2.putText(T,str(preds[0][0])+': Passing', (50, 100), font, 2.5, (255, 255, 255), 2)
                print(str(preds[0][0])+': Passing')
                cv2.imshow('camera',T)
                os.remove('face.jpg')
                #break
            else:
                print('unknow')
                send_email('face.jpg','2743584180@qq.com')
                os.remove('face.jpg')
                import winsound
                duration = 3000  # millisecond
                freq = 440  # Hz
                winsound.Beep(freq, duration)
                #break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
def send_email(img_path,to_list):#发送图片路径，收件人
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.image import MIMEImage
    import smtplib
    import matplotlib.pyplot as plt


    if __name__ == '__main__':
        # 以html格式构建邮件内容
        send_str = '<html><body>'
        send_str += '<center>危险预警-陌生人</center>'
        send_str += '</body></html>'
        pic_path = 'test.png'
        msg = MIMEMultipart()

        # 添加邮件内容
        content = MIMEText(send_str, _subtype='html', _charset='utf8')
        msg.attach(content)

        # 邮件主题
        msg['Subject'] = '陌生人到访'

        # 邮件收、发件人
        user = "2743584180@qq.com"
        msg['To'] = ';'.join(to_list)
        msg['From'] = user
        # 密码（有些邮件服务商在三方登录时需要使用授权码而非密码，比如网易和QQ邮箱）
        passwd = "actvsxsystkpdhag"

        # 构建并添加附件对象
        # 如果不想直接在邮件中展示图片，可以以附件形式添加
        img = MIMEImage(open(img_path, 'rb').read(), _subtype='octet-stream')
        img.add_header('Content-Disposition', 'attachment', filename=pic_path)
        msg.attach(img)

        server = smtplib.SMTP_SSL("smtp.qq.com",port=465)
        server.login(user, passwd)
        server.sendmail(user, to_list, msg.as_string())

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import smtplib
import matplotlib.pyplot as plt


if __name__ == '__main__':
    # 以html格式构建邮件内容
    send_str = '<html><body>'
    send_str += '<center>危险预警</center>'
    
    # html中以<img>标签添加图片，align和width可以调整对齐方式及图片的宽度
    send_str += '</body></html>'
    
    # 画图并保存到本地
    pic_path = 'test.png'
#     plt.plot([1,3,2,4], '--r')
#     plt.title('这是一个测试')
#     plt.savefig(pic_path)

    # 构建message
    msg = MIMEMultipart()
    
    # 添加邮件内容
    content = MIMEText(send_str, _subtype='html', _charset='utf8')
    msg.attach(content)
    
#     # 构建并添加图像对象
#     img1 = MIMEImage(open(pic_path, 'rb').read(), _subtype='octet-stream')
#     img1.add_header('Content-ID', 'image1')
#     msg.attach(img1)

    # 邮件主题
    msg['Subject'] = '这是一封带图邮件'
    
    # 邮件收、发件人
    user = "2743584180@qq.com"
    to_list = "2743584180@qq.com"
    msg['To'] = ';'.join(to_list)
    msg['From'] = user

    # 构建并添加附件对象
    # 如果不想直接在邮件中展示图片，可以以附件形式添加
    img = MIMEImage(open('me.jpg', 'rb').read(), _subtype='octet-stream')
    img.add_header('Content-Disposition', 'attachment', filename=pic_path)
    msg.attach(img)

    # 密码（有些邮件服务商在三方登录时需要使用授权码而非密码，比如网易和QQ邮箱）
    passwd = "actvsxsystkpdhag"
    
    # 登录邮箱
    server = smtplib.SMTP_SSL("smtp.qq.com",port=465)
    server.login(user, passwd)
    print('Login Successful!')
    
    # 发送邮件
    server.sendmail(user, to_list, msg.as_string())
    print('Send Successful')

In [ ]:
#1.0.2

import cv2
import os,dlib,glob
import numpy as np
from skimage import io,transform
import operator

def distMap(frame1, frame2):
    frame1_32 = np.float32(frame1)
    frame2_32 = np.float32(frame2)
    diff32 = frame1_32 - frame2_32
    norm32 = np.sqrt(diff32[:,:,0]**2 + diff32[:,:,1]**2 + \
             diff32[:,:,2]**2)/np.sqrt(255**2 + 255**2 + 255**2)
    dist = np.uint8(norm32*255)
    
    mod = cv2.GaussianBlur(dist, (9,9), 0)
    _, thresh = cv2.threshold(mod, 100, 255, 0)
    _, stDev = cv2.meanStdDev(mod)
    return stDev

# 1.人脸关键点检测器
predictor_path = 'face-model/shape_predictor_68_face_landmarks.dat'
# 2.人脸识别模型
face_rec_model_path = 'face-model/dlib_face_recognition_resnet_model_v1.dat'

# 1.加载正脸检测器
detector = dlib.get_frontal_face_detector()
# 2.加载人脸关键点检测器
sp = dlib.shape_predictor(predictor_path)
# 3. 加载人脸识别模型
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

def get_images_and_labels(faces_folder_path,img_path):#人脸文件夹,需识别的人脸
    # 人脸描述子list
    descriptors = []
    # 对文件夹下的每一个人脸进行:
    # 1.人脸检测
    # 2.关键点检测
    # 3.描述子提取
    candidate = []
    for f in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
        candidate.append(f.split('\\')[1])
        img = io.imread(f)
        if(len(img.shape) != 3):
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        # 1.人脸检测
        dets = detector(img, 1)

        for k, d in enumerate(dets): 
        # 2.关键点检测
            shape = sp(img, d)
        # 3.描述子提取，128D向量
            face_descriptor = facerec.compute_face_descriptor(img, shape)
        # 转换为numpy array
            v = np.array(face_descriptor)  
            descriptors.append(v)

    # 对需识别人脸进行同样处理
    # 提取描述子
    img = io.imread(img_path)
    if(len(img.shape) != 3):
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    dets = detector(img, 1)
    if(len(dets)==0):
        return False
    for k, d in enumerate(dets):
        shape = sp(img, d)
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        d_test = np.array(face_descriptor) 
    
    
    # 计算欧式距离
    dist = []
    for i in descriptors:
        dist_ = np.linalg.norm(i-d_test)
        dist.append(dist_)
    return 1-np.mean(np.array(dist))#正确率
import cv2
cap = cv2.VideoCapture(0)



face = cv2.CascadeClassifier('haarshare/haarcascade_frontalface_default.xml')
while(1):
    ret,frame = cap.read()
    ret2,frame2=cap.read()
    cv2.imshow('camera',frame)
    #动态检测
    stDev = distMap(frame, frame2)
    
    if stDev < 3:
        gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        font = cv2.FONT_HERSHEY_SIMPLEX
        faces = face.detectMultiScale(gray,1.1,3,0,(200,200))
        for (x,y,w,h) in faces:
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2)
            gray_roi = gray[y:y+h,x:x+h]
            #gray_roi=(transform.resize(gray_roi,(64,64))*255).astype(np.uint8)
            cv2.imwrite('face.jpg',gray_roi)
        if(os.path.exists('face.jpg')):

            test = cv2.GaussianBlur(frame, (15,15), 0)
            imgzi = cv2.putText(test,'Testing', (50, 100), font, 2.5, (255, 255, 255), 2)
            cv2.imshow('camera',test)

            preds = {'zhang':0,'mi':0,'wang':0}
            preds['zhang'] = get_images_and_labels('image/zhang','face.jpg')
            preds['mi'] = get_images_and_labels('image/mi','face.jpg')
            #preds['wang'] = get_images_and_labels('image/wang','face.jpg')
            preds = sorted(preds.items(),key = operator.itemgetter(1),reverse = True)
            if(preds[0][1]>0.65):
                T = cv2.GaussianBlur(frame, (15,15), 0)
                imgzi = cv2.putText(T,str(preds[0][0])+': Passing', (50, 100), font, 2.5, (255, 255, 255), 2)
                print(str(preds[0][0])+': Passing')
                cv2.imshow('camera',T)
                os.remove('face.jpg')
                #break
            else:
                print('unknow')
                os.remove('face.jpg')
                #break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
#1.0.1

import os
import cv2
import face_recognition
def get_images_and_labels(path,jpg):
    image_paths = [os.path.join(path,f) for f in os.listdir(path)]
    unknown_image = face_recognition.load_image_file(jpg)
    pred = 0

    for image_path in image_paths:
        known_image = face_recognition.load_image_file(image_path)
        lyf_encoding = face_recognition.face_encodings(known_image)[0]
        try:
            unknown_encoding = face_recognition.face_encodings(unknown_image)[0]
            results = face_recognition.compare_faces([lyf_encoding], unknown_encoding)
            pred += results[0]
        except IndexError as err:
            pred = pred
    pred = pred/len(image_paths)
    return pred
#使用opencv+python实现最简单摄像头实时人脸识别
import cv2
cap = cv2.VideoCapture(0)
face = cv2.CascadeClassifier('haarshare/haarcascade_frontalface_default.xml')
while(1):
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    font = cv2.FONT_HERSHEY_SIMPLEX
    faces = face.detectMultiScale(gray,1.1,3,0,(200,200))
    for (x,y,w,h) in faces:
        img = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2)
        gray_roi = gray[y:y+h,x:x+h]
        cv2.imwrite('face.jpg',gray_roi)
    if(os.path.exists('face.jpg')):
        pred = get_images_and_labels('image/zhang','face.jpg')
        if(pred>0.9):
            imgzi = cv2.putText(frame, 'zhang', (50, 300), font, 1.2, (255, 255, 255), 2)
            print("zhang")
        else:
            imgzi = cv2.putText(frame, 'unknow', (50, 300), font, 1.2, (255, 255, 255), 2)
            print("unknow")
        cv2.imshow('camera',frame)
    else:
        continue
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()